# Favorite Song Selection

### Imports

In [ ]:
from bs4 import BeautifulSoup
import requests
import random as r

## Parameters

### Set Run Mode

In [ ]:
mode = input("Enter the mode: Single or Multiple").lower().replace(" ","")

if mode not in ["single","multiple"]:
    print(f"Entered mode {mode} not valid, proceeding with Single")
    mode = "single"

### Top n songs

In [ ]:
number_of_songs = 1

## Get Album URL from MusicBrainz

In [ ]:
def get_tracklist(url):
    """Gets an Album Tracklist from MusicBrainz. Can only get the first 100 tracks in long albums"""
    
    data = requests.get(url)
    web_data = BeautifulSoup(data.content,"html.parser")

    # Filters the data to obtain track titles
    first_pass = web_data.find_all(class_="title wrap-anywhere")
    second_pass = [result.find_all("a")[0] for result in first_pass]
    track_list = [result.find_all("bdi")[0].text for result in second_pass]

    return track_list

In [ ]:
if mode == "single":
    url = input("Enter the url for the album")
    track_list = get_tracklist(url)

else:
    tracklists = []
    try:
        with open("albums.txt","r") as album_links:
            albums = album_links.readlines()
    except FileNotFoundError:
        raise FileNotFoundError("Album URL file albums.txt does not exist. If only using one album URL use mode 'Single' instead")
        
    # Build a list of tracks for each album supplied through the file
    for album_url in albums:
        tracklists.append(get_tracklist(album_url.rstrip("\n")))

## Pick Favorite Song

In [ ]:
def favorite_song(tracks):
    available = [song for song in tracks]

    for i in range(len(available)-1):
        choices = r.sample(available,2)

        selection = input(f"{choices[0]} vs {choices[1]} (A vs B): ").upper()

        if selection == "A":
            available.remove(choices[1])
        elif selection == "B":
            available.remove(choices[0])
        else:
            raise ValueError(f"Invalid Input ({selection}): Enter either A or B. ")
        
    return available[0]

In [ ]:
def top_n_songs(fav_song,tracks,n):
    song_list = [song for song in tracks]
    favorite_songs = []
    for song in range(n):
        nth_fav = fav_song(song_list)
        song_list.remove(nth_fav)
        favorite_songs.append(nth_fav)
        print(f"#{song+1} song: {nth_fav}") # Print out song choices as they are made
    
    for index,val in enumerate(favorite_songs):
        print(f"{index+1} : {val}") # Print out song ranking
    
    return favorite_songs

## Choose Top n songs

In [ ]:
if mode == "single":
    top_n_songs(favorite_song,track_list,n = number_of_songs)

else:
    with open("playlist.txt","a+") as file:
        for track_list in tracklists:
            favorite_songs = top_n_songs(favorite_song,tracks = track_list,n = number_of_songs)
            for song in favorite_songs:
                file.write(f"{song}\n")